In [ ]:
!pip install soynlp

In [ ]:
!pip install hdbscan

In [2]:
import pandas as pd
import numpy as np
data = pd.read_csv('./data/accident_df_ver2.csv')
# 중복 및 null 값 제거
data = data.drop_duplicates('body_prep', keep = 'first').reset_index(drop=True)
data = data.dropna().reset_index(drop=True)
bodys = [txt for id, txt in enumerate(data['body_prep'])]
title = [txt for id, txt in enumerate(data['title'])]

## - 전처리

### NewsNounExtractor
* 뉴스데이터에서 좋은 성능을 내는 명사추출기

In [213]:
from soynlp.noun import NewsNounExtractor

noun_extractor = NewsNounExtractor(
    max_left_length=13, 
    max_right_length=10,
    predictor_fnames=None,
    verbose=True
)

used default noun predictor; Sejong corpus based logistic predictor
C:/ProgramData/Anaconda3/lib/site-packages/soynlp
local variable 'f' referenced before assignment
local variable 'f' referenced before assignment


In [214]:
nouns = noun_extractor.train_extract(bodys)

scan vocabulary ... 
done (Lset, Rset, Eojeol) = (228087, 131196, 116728)
predicting noun score was done                                        
before postprocessing 54829
_noun_scores_ 12594
checking hardrules ... done0 / 12594+(이)), NVsubE (사기(당)+했다) ... done
after postprocessing 8673
extracted 166 compounds from eojeolss ... 17000 / 17210

In [215]:
nouns

{'인정된다고': NewsNounScore(score=0, frequency=8, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '2시50분쯤': NewsNounScore(score=0, frequency=3, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '변호인도': NewsNounScore(score=0, frequency=7, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '코로나바이러스': NewsNounScore(score=0, frequency=503, feature_proportion=0.0, eojeol_proportion=0.9980119284294234, n_positive_feature=0, unique_positive_feature_proportion=0),
 '고속단정': NewsNounScore(score=0, frequency=3, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '4층짜리': NewsNounScore(score=0, frequency=7, feature_proportion=0, eojeol_proportion=1.0, n_positive_feature=0, unique_positive_feature_proportion=0),
 '11시20분쯤': NewsNounScore(score=0, frequency=6, feature_proportion=0,

In [216]:
# score 점수에 따라서 분류
# 빈도수 * 명사점수 기준

sorted_nouns = sorted(nouns.items(), 
    key=lambda x:-x[1].frequency * x[1].score)

sorted_by_score = {}

for i, (word, score) in enumerate(sorted_nouns):
    if i % 5 == 0:
        print()    
    if score.score > 0.4:
        sorted_by_score[word] = score.score
        print('%6s (%.2f)' % (word, score.score), end='')


    지난 (1.00)   있다. (1.00)    혐의 (0.94)    수사 (0.94)   A씨는 (1.00)
    오후 (0.96)    오전 (0.97)    관련 (0.85)    사건 (0.66)   코로나 (0.70)
    신고 (0.86)    사고 (0.85)    발견 (0.99) 코로나19 (0.97)   지난달 (1.00)
    당시 (0.73)    검사 (0.89)    위반 (0.94)   피해자 (0.73)    현장 (0.92)
    여성 (0.65)    함께 (0.99)    이후 (0.96)    재판 (0.81)    이날 (0.43)
    자신 (0.95)    선고 (0.95)    검찰 (0.44)    모두 (1.00)    인근 (0.94)
    상태 (0.78)    병원 (0.93)   확진자 (0.56)    사실 (0.63)    예정 (0.98)
    때문 (1.00)    해당 (0.98)    파악 (0.97)    과정 (0.96)    가능 (1.00)
    B씨 (0.41)    차량 (0.69)    추가 (1.00)    기소 (1.00)    남성 (0.64)
    변호 (0.89)    징역 (0.98)   감염증 (1.00) 감염증(코 (1.00)   마스크 (0.92)
    범죄 (0.79)    이상 (0.91)    집회 (0.90)    사이 (0.94)    처벌 (0.75)
    범행 (0.64)    최근 (1.00)감염증(코로나19) (0.95)    경우 (0.68)    방역 (0.77)
    조치 (0.91)   뉴스1 (1.00)    불법 (1.00)    시장 (0.58)   아파트 (0.84)
    치료 (0.98)    고발 (0.92)    고소 (0.96)    의혹 (0.75)    2명 (0.61)
    지역 (0.72)    휴대 (0.99)    화재 (0.81)    압수 (0.98)    사망 (0.58)
    뉴

In [219]:
# tokenizer생성

from soynlp.tokenizer import LTokenizer
tokenizer = LTokenizer(scores=final_nouns_list)

tok_sent = []
for i in range (len(bodys)):
    sent = bodys[i]
    tok_sent.append(tokenizer.tokenize(sent))

In [220]:
noun_sent = []
for doc in tok_sent:
    noun = [word for word in doc if word in final_nouns_list]
    noun_sent.append(noun)

In [221]:
corpus = [" ".join(doc) for doc in noun_sent]
corpus[:5]
# for doc in tok_sent:
#     if word in sorted_by_score:
#         print(word)
#         print(noun)
#     final.append

['아산 유성기업 분쟁 마침표 유성기업 아산 공장 있다. 민주노총 금속 충남도 유성기업 합의안 조합원 찬성 임금 파업 과정 고소 주요 내용 합의안 조합원들 합의안 현장 설치 감시 카메라 철거 재발 방지 쌍방 유감 차별 금지 포함 중순 공식 갖기 유성기업 투쟁 마음 회사 유성기업 연속 조항’ 불이행 이유 파업 아산 물론 충북 영동 카드 정부 공권력 농성 강제 해산 이후 고소 과정 폭력 설립 조합원 임원 사건 사건 연루 조합원 실형 선고 유성기업 회장 배임 횡령 혐의 지난 법정 수감 교섭 합의안 찬성 지난 교섭 재개 유성기업 목표 의견 실무 대표 교섭 지난 서명 합의안 전국 관계자들 지난 기자회견 배임 혐의 기소 유성기업 회장 법정 선고 있다. 유성기업 충남도 오랜 갈등 직원 심리 지원 방침 유성기업 행정 지원 나서기 대표 자동 유성기업 진심 “그동안 불명 상징 기업 거듭',
 '영화 화살 정지 감독 제작 스태프 지원 횡령 혐의 검찰 넘겨졌다 감독 영화 아우라픽처스 업무 횡령 일부 혐의 기소 의견 이달 검찰 감독 아우라픽처스 업무 관리 법률 위반 혐의 고발 감독 영화 스태프 처우 개선 목적 화살 제작 아우라픽처스 지원 스태프 통장 프로듀서 계좌 횡령 제작 과정 일부 스태프 급여 제작 대표 계좌 방식 횡령 변호 감독 경우 업무 횡령 혐의 기소 사기 보조 관리 위반 불기소 의견 혐의 모두 불기소 업무 횡령 감독 한편 여자 감독 전쟁 화살 사회적 이슈 영화',
 '아동 성범죄자 오전 경기도 안산 안산준법지원센터 있다. 아동 성폭행 바깥 안산준법지원센터 경기 지난 모습 직후 시간대 거주지 인근 마트 사실 CCTV 경비초소 전자 관련기관 전담 보호 감시 감시 관련 안산준법지원센터 관찰 빈틈 이뤄지고 “주 불안 염려 시간대 CCTV 경비초소 설치 있다. 안산준법지원센터 전담 특별 경기남부 기동대 제대 감시 있다.',
 '국내 외제차 안전 평가 생산 가장 안전 \'아 평가 충돌 보행자 안전 사고 예방 안전 분야 충돌 정면 부분 측면 충돌 안전 좌석 안전 에어백 장치 안전 항목

In [222]:
# Word Embedding
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.preprocessing import normalize

# Extracting features from text files
count_vect = TfidfVectorizer(min_df = 2, # 토큰이 나타날 최소 문서 개수로 오타나 자주 나오지 않는 특수한 전문용어 제거에 좋다. 
                            #  ngram_range=(1, 3),
                             max_features = 90000
                             )
tfidf_transformer = TfidfTransformer()

cnt_vector = count_vect.fit_transform(corpus)
tfidf_vector = tfidf_transformer.fit_transform(cnt_vector)
tfidf_vector.shape

(2508, 6008)

In [223]:
len(corpus[0])

433

## - HDBSCAN

In [302]:
# Modeling
def hdbscancluster(data, corpus_vector, min_samples):
    
    import hdbscan
    from collections import Counter
    
    hdb = hdbscan.HDBSCAN(min_samples=min_samples, alpha=0.5)  # min_cluster_size=2,
    hdbscan_model = hdb.fit(corpus_vector)

    labels1 = hdbscan_model.labels_
    n_clusters_ = len(set(labels1)) - (1 if -1 in labels1 else 0) # Number of clusters in labels
    # print(labels1)
    print('news documents:      {}'.format(len(labels1)))
    print('not clustering docs: {}'.format(Counter(labels1==-1)[1]))
    print('number of clusters:  {}'.format(n_clusters_)) # number of clusters
    
    clusters = {} # a dictionary for different cluster 
    for c, i in enumerate(labels1):
        if i == -1:
            continue
        elif i in clusters:
            clusters[i].append(data[c])
        else:
            clusters[i] = [data[c]]
    return clusters, labels1

In [305]:
for c in h_clusters:
    print("Cluster No."+" "+str(c)+" 문서수: "+str(len(h_clusters[c])))
    for news in h_clusters[c]:
        print(news)
    print()

Cluster No. 29 문서수: 10
조두순 보름만에 첫 외출...신호 울린 순간, 경찰 뒤쫓았다
조두순 호송차 올라가 발로 부순 유튜버 사전구속영장
조두순 얼굴·거주지 담긴 신상정보 '성범죄자 알림e'에 떴다
안산집 도착한 조두순, 말 한마디 없이 집으로 사라졌다
뒷짐진채 90도 숙인 조두순 "이렇게 많이 모일줄 몰랐다"
조두순 지역구 김남국 “더 중요한 건 피해자 인권, 사회 안전”
관용차 타고, 오른손에 귤 하나 들고 안산 온 조두순
"조두순 악몽 12년째, 영구격리 지켜달라" 피해자父의 호소
이재정 경기교육감 "조두순 경계 당연하지만, 따뜻한 사회가 됐으면…"
선관위, 사전투표지 촬영·공개·훼손 3명 고발…“선거질서 교란 강력 대응”

Cluster No. 129 문서수: 27
풀려난 전광훈 "민주당은 천만만콩떡···대한민국이 이겼다"
[속보] 檢, '특정정당 지지' 전광훈에 징역 2년6개월 구형
경찰, 사랑제일교회 목사 등 2명 구속영장…CCTV 등 은폐한 혐의
126만명에 "광화문집회 오라" 문자, 사랑제일교회 아이디로 발송
"광복절 집회 참석하라" 126만명에 문자 뿌린 사랑제일교회
전광훈 보석 취소, 140일만에 재구속…보석금 3000만원 몰취
전광훈 측 "3일 오후 사택 압수수색 등에 대한 입장 밝힐 것"
[속보] 靑 '전광훈씨'라고 부른날...경찰, 교회 사택 압수수색
부산경찰 광화문 집회 명단 제출 거부한 인솔자 압수수색
'전광훈 입' 강연재 "교인 코로나 취급"…대규모 소송 꺼냈다
서울 이어 전주서도 '광화문 인솔' 목사 7명 압수수색…사랑제일교회 연관성 '촉각'
경찰, 사랑제일교회 압수물 분석…"PC 내 교인 자료 포렌식"
4시간여만에 박스 8개···사랑제일교회 압수수색 한밤 종료
文 "공권력 보여줘라" 경고한 날, 사랑제일교회 압수수색
[속보] 경찰, 사랑제일교회 압수수색…교인 명단 확보 시도
사랑제일교회 12시간 밤샘대치…찬송가 부르며 명단확보 반발
광화문 집회 명단 요청 묵살한 목사…잠적 하루만에 경찰 출두
울산서도 '광복절 집